In [1]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
TRAIN_TEXT_FILE_PATH = 'sample_data/train_text.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'е', 'а', 'и', 'н', 'т', 'с', 'л', 'в', 'р', 'к', 'м', ',', 'д', 'у', 'п', 'ь', 'я', 'ч', 'ы', 'г', 'з', 'б', '.', 'й', 'ж', 'х', 'ш', '—', 'ю', '\n', 'щ', 'ц', 'э', 'А', 'П', '-', '!', 'Н', '?', '«', '»', 'И', 'В', ';', 'К', 'С', 'Д', 'Е', 'ф', ':', 'О', 'Л', 'Т', 'М', 'Э', 'ё', 'Я', 'Ф', 'Р', 'Г', '(', ')', 'Ч', 'ъ', 'Б', 'У', 'З', '`', "'", 'Щ', 'Х', '“', 'Ж', '„', 'Ц', '[', '*', ']', 'Ь', 'Ы', 'Й', 'Ш']


In [3]:
SEQ_LEN = 256
BATCH_SIZE = 32

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [4]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=300, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [5]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [11]:
%time
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=3)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.3
)

n_epochs = 50000 #50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 45:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
Loss: 3.313232904010349
  с а а в  о  о , ос ааи и н н р и   в а е о  а     ои аа н ин ио и н м  во н м ие о  о и и п  ео   е в и оот  л ут о а и  с  ко  ооссао л н    с и  о оол о о а т и  иеооов н  о оа ио   т о е о н    т а е ое с атоа    те и  м вн ооо е и  е т р т и  оа н  о  аа си и е к а о в о    е а и оаое оат е т р
Loss: 2.980644628736708
 толек с токи сналоло талола доно но но тал мотело тем в татола но ста токо натол сго потеве стет насто ни всте потонола нет нола папелиге тот но стеото ни состамо то снова петоно нололи сто стох довто кете новало то титело с вевите повонос потовот стено вротолоно толо сто сролеста зом стесво не тотн
Loss: 2.6451421366797554
 сесто полаком не та но на в тела в тот и секи вата боротаром столо сетол дотил вала на провала статась пекет каде но на то на сла но сатас делу оссототела том на то но доте востала раколини в как поролирора на на столи сотасто нано сно сна потолола на не на 

In [14]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.4, 
    prediction_len=1000, 
    start_text='Печаль '
    )
)

Печаль просто места.
 — Ну, вот то же самое и со мной, — сказал Санчо: — глядя на моего господина, я почувствовал то, что должен был чувствовать он, когда перекатывался с выступа на выступ.
 — А как зовут твоего господина? — спросила Мариторна.
 — Дон-Кихотом Ламанчским. Это один из славнейших странствующих рыцарей, когда-либо существовавших на свете, — с гордостью ответил Санчо.
 — Странствующий рыцарь? — с удивлением повторила служанка. — Что же это значит?
 — Э, как ты глупа, если даже и этого не знаешь, моя милая! Странствующий рыцарь — это... как бы тебе сказать?.. это такой человек, который каждый день может ожидать или императорской короны, или... хорошей трепки. Сегодня он самый жалкий человек в мире, а завтра, глядишь, он повелитель над несколькими королевствами, из которых любое может уступить своему оруженосцу. Поняла?
 — И ты его оруженосец? — спросила хозяйка.
 — Конечно! Разве вы этого не видите?
 — Не привычны мы к таким гостям, потому и не угадали сразу... Ну, и что же,

In [15]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=1000, 
    start_text='Печаль '
    )
)

Печаль поводы мы у нее самого его касаются! Господи, хорошо еще, что он идиот и... и... друг дома! Только неужели ж Аглая прельстилась на такого уродика! Господи, что я плету! Тьфу! Оригиналы мы... под стеклом надо нас всех показывать, меня первую, по десяти копеек за вход. Не прощу я вам этого, Иван Федорыч, никогда не прощу! И почему она теперь его не шпигует? Обещалась шпиговать и вот не шпигует! Вон, вон, во все глаза на него смотрит, молчит, не уходит, стоит, а сама же не велела ему приходить... Он весь бледный сидит. И проклятый, проклятый этот болтун Евгений Павлыч, всем разговором один завладел! Ишь разливается, слова вставить не дает. Я бы сейчас про всё узнала, только бы речь навести... ».
 Князь и действительно сидел, чуть не бледный, за круглым столом и, казалось, был в одно и то же время в чрезвычайном страхе и, мгновениями, в непонятном ему самому и захватывающем душу восторге. О, как он боялся взглянуть в ту сторону, в тот угол, откуда пристально смотрели на него два зна